In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import Adam

In [2]:
# 1727, 191
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/1727-nonsquare-train-from-1918-nonsquare-spmm-over-3s.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/191-nonsquare-test-from-1918-nonsquare-spmm-over-3s.csv')

In [3]:
# feature 1개 추가

# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['bz_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['bz_smdm']

In [4]:
#  feature 4개 추가 (xgbregressor 용 feature)

# Train + Valid
X_train_v2 = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz','lnnz+rnnz','lnnz+lc*rc','lr*lc+lc*rc']] 

# Test
X_test_v2 = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz','lnnz+rnnz','lnnz+lc*rc','lr*lc+lc*rc']] 

In [5]:
# 데이터 표준화(Standardization)
from sklearn.preprocessing import StandardScaler

# 변형 객체 생성
std_scaler = StandardScaler()

# 훈련데이터의 모수 분포 저장
std_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train_scaled = std_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test_scaled = std_scaler.transform(X_test)

In [6]:
from sklearn.metrics import mean_squared_error

def mape_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def rmse_error(y_true, y_pred):
    rmse = np.sqrt(np.mean(np.square(y_pred - y_true))) 
    return rmse

### Xgbregressor

In [7]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
xgbregressor_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(14),
learning_rate=0.02398298537228648,
n_estimators=int(149),
subsample=0.5732030368686496,
reg_lambda = 0.36548495335549436,    
n_jobs=-1
                             )
# 모델 훈련
xgbregressor_model.fit(X_train_v2, y_train)

XGBRegressor(learning_rate=0.02398298537228648, max_depth=14, n_estimators=149,
             n_jobs=-1, objective='reg:squarederror',
             reg_lambda=0.36548495335549436, subsample=0.5732030368686496)

### dnn

In [8]:
# 모델 생성
def build_model():

    model=Sequential()

    model.add(Dense(256, activation="relu", input_shape=(X_train.shape[1],)))  
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1))
    
    optimizer = Adam(lr=0.01)
    
    model.compile(optimizer=optimizer ,
                  loss='mape',
                  metrics=['mape'])
    return model

dnn_model = build_model()

In [9]:
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_mape', patience=150)

EPOCHS = 100000

dnn_model.fit(X_train_scaled, 
                y_train,
                epochs=EPOCHS, 
                validation_split = 0.1, 
                verbose =0, 
                callbacks=[early_stop, PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
...................

### rfr

In [10]:
from sklearn.ensemble import RandomForestRegressor

rfr_model = RandomForestRegressor(
criterion='mse',
max_depth=24,
min_samples_leaf=2, 
min_samples_split=4, 
n_estimators=800
)

rfr_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=24, min_samples_leaf=2, min_samples_split=4,
                      n_estimators=800)

### test

In [11]:
xgbregressor_y_pred = xgbregressor_model.predict(X_test_v2)
dnn_y_pred = dnn_model.predict(X_test_scaled).reshape(-1,)
rfr_y_pred = rfr_model.predict(X_test)

In [12]:
print(mape_error(y_test,xgbregressor_y_pred))
print(mape_error(y_test,dnn_y_pred))
print(mape_error(y_test,rfr_y_pred))

12.307202381784455
13.613920626331275
15.30806246327954


In [13]:
result_list = {}
mape_list = np.array([])

# 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
for idx,value in enumerate(y_test):
    mape_temp = {}
    median_temp = np.array([])
    
    # 각 모델의 예측값
    xgbregressor_predicate = int(xgbregressor_y_pred[idx])
    dnn_predicate = int(dnn_y_pred[idx])
    rfr_predicate = int(rfr_y_pred[idx])
    
    # 각 모델의 예측값을 배열에 삽입
    median_temp = np.append(median_temp, np.array([xgbregressor_predicate,dnn_predicate,rfr_predicate]))
    
    # median 값 사용
    mape = abs((value - np.median(median_temp)) / value) * 100
    
    # mean 값 사용
    # mape = abs((value - np.mean(median_temp)) / value) * 100
    
    mape_temp['xgb_pred'] = xgbregressor_predicate
    mape_temp['dnn_pred'] = dnn_predicate
    mape_temp['rfr_pred'] = rfr_predicate
    mape_temp['best_pred'] = np.median(median_temp)
    mape_temp['real'] = value
    mape_temp['mape'] = mape

    mape_list = np.append(mape_list,np.array([mape]))
    result_list[idx] = mape_temp

result_list_sort = sorted(result_list.values(), key=lambda x:(x['mape']), reverse=True)
result_list_sort  

[{'xgb_pred': 31350,
  'dnn_pred': 28232,
  'rfr_pred': 33333,
  'best_pred': 31350.0,
  'real': 13334,
  'mape': 135.1132443377831},
 {'xgb_pred': 10863,
  'dnn_pred': 9374,
  'rfr_pred': 12179,
  'best_pred': 10863.0,
  'real': 5543,
  'mape': 95.97690781165434},
 {'xgb_pred': 12014,
  'dnn_pred': 13159,
  'rfr_pred': 13424,
  'best_pred': 13159.0,
  'real': 7369,
  'mape': 78.57239788302348},
 {'xgb_pred': 8688,
  'dnn_pred': 8209,
  'rfr_pred': 7767,
  'best_pred': 8209.0,
  'real': 4686,
  'mape': 75.18139137857447},
 {'xgb_pred': 11447,
  'dnn_pred': 10475,
  'rfr_pred': 11062,
  'best_pred': 11062.0,
  'real': 6487,
  'mape': 70.52566671805148},
 {'xgb_pred': 16447,
  'dnn_pred': 18748,
  'rfr_pred': 16597,
  'best_pred': 16597.0,
  'real': 10530,
  'mape': 57.616334283000946},
 {'xgb_pred': 56638,
  'dnn_pred': 44586,
  'rfr_pred': 67593,
  'best_pred': 56638.0,
  'real': 36559,
  'mape': 54.922180584808125},
 {'xgb_pred': 66179,
  'dnn_pred': 65815,
  'rfr_pred': 72811,
  'bes

In [14]:
# 최적의 pred을 골라냈을 때 mape 평균
print(np.mean(mape_list))

12.499723415003894
